In [7]:
#import libraries
import pandas as pd
import yfinance as yf
import hvplot.pandas
import panel as pn
import numpy as np
from pandas_datareader import data as pdr

In [8]:
#read in csv
x = pd.read_csv("SPY_2020.csv")

In [9]:
#look at the data
x.head()

,underlying,underlying_last,exchange,optionroot,optionext,type,expiration,quotedate,strike,last,...,volume,openinterest,impliedvol,delta,gamma,theta,vega,optionalias,IVBid,IVAsk
0,SPY,324.87,*,SPY200103C00260000,NaN,call,01/03/2020,01/02/2020,260.0,53.48,...,0,0,0.1607,1.0,0.0,-4.9722,0.0,SPY200103C00260000,0.0,3.3567
1,SPY,324.87,*,SPY200103P00260000,NaN,put,01/03/2020,01/02/2020,260.0,0.00,...,0,4239,0.2574,0.0,0.0,0.0000,0.0,SPY200103P00260000,0.0,2.4991
2,SPY,324.87,*,SPY200103C00265000,NaN,call,01/03/2020,01/02/2020,265.0,0.00,...,0,0,0.1607,1.0,0.0,-5.0678,0.0,SPY200103C00265000,0.0,3.1326
3,SPY,324.87,*,SPY200103P00265000,NaN,put,01/03/2020,01/02/2020,265.0,0.01,...,0,32850,0.2574,0.0,0.0,0.0000,0.0,SPY200103P00265000,0.0,2.3017
4,SPY,324.87,*,SPY200103C00270000,NaN,call,01/03/2020,01/02/2020,270.0,0.00,...,0,0,0.1607,1.0,0.0,-5.1634,0.0,SPY200103C00270000,0.0,2.8806


In [10]:
#drop unnecessary columns
x.drop(columns=['optionroot','optionext','impliedvol','delta','gamma','theta','vega',
                'optionalias','IVBid','IVAsk','openinterest','bid','ask'],inplace=True)

In [11]:
#calc and insert column 'days to expiry'
x[['expiration','quotedate']] = x[['expiration','quotedate']].apply(pd.to_datetime) #if conversion required
x['days_to_expiry'] = (x['expiration'] - x['quotedate']).dt.days

In [12]:
#ensure column 'underlying_last' and strike are set to data type integer
x['underlying_last']=x['underlying_last'].astype(int)
x['strike']=x['strike'].astype(int)

In [13]:
x['last_trading_day'] = x['expiration'] - pd.Timedelta(1, unit='D') 

In [14]:
#clean dataframe 
# index_names = x[ x['days_to_expiry'] != 30 ].index 
index_names = x[x['last'] == 0].index
x.drop(index_names, inplace = True) 

#clean dataframe so we only observe at-the-money options
index_names2 = x[ x['strike'] != (x['underlying_last'])].index
x.drop(index_names2, inplace = True) 

#sort and reset
x.sort_values(by = ['type','expiration']).reset_index().head()

,index,underlying,underlying_last,exchange,type,expiration,quotedate,strike,last,volume,days_to_expiry,last_trading_day
0,162,SPY,324,*,call,2020-01-03,2020-01-02,324,1.20,67169,1,2020-01-02
1,164,SPY,324,*,call,2020-01-03,2020-01-02,324,0.77,24561,1,2020-01-02
2,7496,SPY,322,*,call,2020-01-03,2020-01-03,322,0.67,38277,0,2020-01-02
3,7498,SPY,322,*,call,2020-01-03,2020-01-03,322,0.25,35744,0,2020-01-02
4,314,SPY,324,*,call,2020-01-06,2020-01-02,324,1.50,50887,4,2020-01-05


In [15]:
# pull in spy historical price data
spy = yf.Ticker("SPY")
yf.pdr_override()
y = pdr.get_data_yahoo('SPY',start ='2020-01-01', end ='2020-12-31').drop(columns=['Adj Close']).reset_index()
y[['Date']] = y[['Date']].apply(pd.to_datetime)
y.drop(columns=['Open','High','Low','Volume'], inplace=True)
y.head()

[*********************100%***********************]  1 of 1 completed


,Date,Close
0,2020-01-02,324.869995
1,2020-01-03,322.410004
2,2020-01-06,323.640015
3,2020-01-07,322.730011
4,2020-01-08,324.450012


In [16]:
#rename columns and reorder for merge
x.rename(columns={'last_trading_day':'Date'},inplace=True)
x= x[['Date', 'underlying', 'underlying_last', 'type', 'quotedate', 'strike', 'last', 'volume', 'days_to_expiry']]
x.head()

,Date,underlying,underlying_last,type,quotedate,strike,last,volume,days_to_expiry
162,2020-01-02,SPY,324,call,2020-01-02,324,1.20,67169,1
163,2020-01-02,SPY,324,put,2020-01-02,324,0.28,56412,1
164,2020-01-02,SPY,324,call,2020-01-02,324,0.77,24561,1
165,2020-01-02,SPY,324,put,2020-01-02,324,0.44,14515,1
314,2020-01-05,SPY,324,call,2020-01-02,324,1.50,50887,4


In [50]:
#calculate in closing price to existing DF 
calls = pd.merge(x, y, how='left', on='Date')
calls['Options_Implied_Price'] = calls['strike'] + calls['last']
calls['last'].dropna()
put_index_names = calls[calls['type'] == 'put'].index
calls.drop(put_index_names, inplace = True) 


puts =  pd.merge(x, y, how='left', on='Date')
puts['Options_Implied_Price'] = (puts['strike'] - puts['last'])
puts['last'].dropna()
call_index_names = puts[puts['type'] == 'call'].index
puts.drop(call_index_names, inplace = True) 

z = pd.concat([calls,puts],axis=0, sort=True).dropna().reset_index()

z.head()

,index,Close,Date,Options_Implied_Price,days_to_expiry,last,quotedate,strike,type,underlying,underlying_last,volume
0,0,324.869995,2020-01-02,325.20,1,1.20,2020-01-02,324,call,SPY,324,67169
1,2,324.869995,2020-01-02,324.77,1,0.77,2020-01-02,324,call,SPY,324,24561
2,6,322.730011,2020-01-07,325.86,6,1.86,2020-01-02,324,call,SPY,324,6467
3,8,326.649994,2020-01-09,326.25,8,2.25,2020-01-02,324,call,SPY,324,18769
4,12,327.450012,2020-01-14,326.75,13,2.75,2020-01-02,324,call,SPY,324,2072


In [63]:
#visualize data
scatter_plot = z.hvplot.scatter(
x='Date',
y='Close',
ylabel='Price',
color='red')

boxplot = z.hvplot.violin(y='Options_Implied_Price', by='Date', height=400, width=1000, legend=False, title='Options Implied Prices vs. Closing Prices', rot=90)
plot_1 = boxplot * scatter_plot

In [57]:
scatter_plot2 = z.hvplot.scatter(
x='Date',
y='Options_Implied_Price',
ylabel='Price',
color='type')

line_plot = z.hvplot.line(
x='Date',
y='Close',
ylabel='Price',
color='red',
height=400, 
width=1000,
title='SP500 Closing Prices vs. Monthly Calls and Puts')

plot_2 = scatter_plot2 * line_plot

In [58]:
#calc mean options implied price and compare to timeseries data of closing spy prices
mean_values = z.groupby(z.Date.dt.date).mean()
mean_values

plot_3 = mean_values.hvplot.line(
x='Date',
y=['Close','Options_Implied_Price'],
ylabel='Price',
)

In [59]:
#use bivariate/density plot to visualize the relationship between implied price and the degree and pattern of relation between the two variables.

plot_4 = z.hvplot.bivariate('Close', 'Options_Implied_Price', legend=False, width=1000, height=400, title='Correlation Surface Plot')

In [62]:
# # Create a Title for the Dashboard
title = '#Options and Stock Prices'

# # Create a tab layout for the dashboard
cover_page = pn.Column(title)

tab_1 = pn.Column(
     "## Timeseries: Implied prices vs. actual",
    plot_2,
    plot_3,
)

tab_2 = pn.Column(
     "## Monthly Options as Predictors of Future Stock Prices",
    plot_1,
    plot_4,
)

dashboard = pn.Tabs(
     ("Cover Page",cover_page),
     ("Historical Data",tab_1),
     ("Visualizing Predictive Power of Options",tab_2),
 )

# # Serve the# dashboard
dashboard.servable()

Tabs
    [0] Column
        [0] Markdown(str)
    [1] Column
        [0] Markdown(str)
        [1] HoloViews(Overlay)
        [2] HoloViews(NdOverlay)
    [2] Column
        [0] Markdown(str)
        [1] HoloViews(Overlay)
        [2] HoloViews(Bivariate)